In [1]:
from pymongo import MongoClient
import pandas as pd
from datetime import datetime

In [24]:
from datetime import datetime

# Get today's date
today_date = datetime.today().strftime('%Y-%m-%d')

print(today_date)


2024-08-05


In [30]:
MONGO_URI = 'mongodb://localhost:27017/'  # Update with your MongoDB URI
DATABASE_NAME = 'E2E_TESTING'
TOPIK = 'INSTANSI'

client_db = MongoClient(MONGO_URI)
db = client_db[DATABASE_NAME]
topik_collection = db[TOPIK]

document = topik_collection.find_one({'_id': 'news'})

data = document['data'] if document else []
# Convert to DataFrame
df = pd.DataFrame(data)
sentiment_counts = df['sentimen'].value_counts(normalize=True) * 100
pie_sentiment = {
    "labels": sentiment_counts.index.tolist(),
    "values": sentiment_counts.values.tolist()
}

df_pie = pd.DataFrame(pie_sentiment)
df_pie['keyword'] = TOPIK
df_pie['createdAt'] = datetime.today().strftime('%Y-%m-%d')

buzzer_counts = df['buzzer'].value_counts(normalize=True) * 100
pie_buzz = {
    "labels": buzzer_counts.index.tolist(),
    "values": buzzer_counts.values.tolist()
}

df_piebuz = pd.DataFrame(pie_buzz)
df_piebuz['keyword'] = TOPIK
df_piebuz['createdAt'] = datetime.today().strftime('%Y-%m-%d')


result = {'pie_sentiment' : df_pie.to_dict('records'),
          'pie_buzz' : df_piebuz.to_dict('records')}

In [31]:
result

{'pie_sentiment': [{'labels': 'negatif',
   'values': 66.66666666666666,
   'keyword': 'INSTANSI',
   'createdAt': '2024-08-05'},
  {'labels': 'positif',
   'values': 33.33333333333333,
   'keyword': 'INSTANSI',
   'createdAt': '2024-08-05'}],
 'pie_buzz': [{'labels': 'not_buzzer',
   'values': 59.25925925925925,
   'keyword': 'INSTANSI',
   'createdAt': '2024-08-05'},
  {'labels': 'buzzer',
   'values': 40.74074074074074,
   'keyword': 'INSTANSI',
   'createdAt': '2024-08-05'}]}

# ===========

In [35]:
# Check for existing 'news_ep' document
news_ep_document = topik_collection.find_one({'_id': 'news_ep'})

# Function to check for duplicate entries
def is_duplicate(record, existing_data):
    for entry in existing_data:
        if 'keyword' in entry and 'createdAt' in entry:
            if entry['keyword'] == record['keyword'] and entry['createdAt'] == record['createdAt']:
                return True
    return False

# Update or insert new data
if news_ep_document:
    # Check for duplicates in 'pie_sentiment'
    new_pie_sentiment = [
        item for item in result['pie_sentiment']
        if not is_duplicate(item, news_ep_document.get('pie_sentiment', []))
    ]
    
    # Check for duplicates in 'pie_buzz'
    new_pie_buzz = [
        item for item in result['pie_buzz']
        if not is_duplicate(item, news_ep_document.get('pie_buzz', []))
    ]
    
    # Update the document if there are new items
    if new_pie_sentiment or new_pie_buzz:
        update_fields = {}
        if new_pie_sentiment:
            update_fields['pie_sentiment'] = news_ep_document.get('pie_sentiment', []) + new_pie_sentiment
        if new_pie_buzz:
            update_fields['pie_buzz'] = news_ep_document.get('pie_buzz', []) + new_pie_buzz
        
        topik_collection.update_one(
            {'_id': 'news_ep'},
            {'$set': update_fields}
        )
else:
    # Insert new 'news_ep' document
    topik_collection.insert_one({'_id': 'news_ep', **result})

print("Data checked for duplicates and inserted/updated successfully.")

Data checked for duplicates and inserted/updated successfully.


In [ ]:
existing_document = data_collection.find_one({'_id': 'news'})

# Check for duplicates based on 'url' and append new data
if existing_document and 'data' in existing_document:
    existing_data = existing_document['data']
    existing_urls = {item['url'] for item in existing_data}
    
    # Filter out new items that already exist based on the 'url' field
    new_data = [item for item in data_json if item['url'] not in existing_urls]
    
    if new_data:
        existing_data.extend(new_data)
        data_collection.update_one(
            {'_id': 'news'},
            {'$set': {'data': existing_data}}
        )
else:
    data_collection.update_one(
        {'_id': 'news'},
        {'$set': {'data': data_json}},
        upsert=True
    )
print("data News Saved!")

# =================

In [37]:
document = topik_collection.find_one({'_id': 'news_ep'})


In [45]:
document['pie_sentiment']

[{'labels': 'negatif',
  'values': 55.55555555555556,
  'keyword': 'INSTANSI',
  'createdAt': '2024-08-05'},
 {'labels': 'positif',
  'values': 44.44444444444444,
  'keyword': 'INSTANSI',
  'createdAt': '2024-08-05'}]

In [40]:
type(document['pie_sentiment'])

list